<a href="https://colab.research.google.com/github/apotato369550/hungry-markov-model/blob/main/hungry_markov_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Markov Model Project (Attempt #2)

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf
import tensorflow_probability as tfp


## Declaring States

In [ ]:
state = {
    0: "Pizza",
    1: "Stew",
    2: "Cookies",
    3: "Nothing",
    4: "Ice Cream"
}
print(state)

{0: 'Pizza', 1: 'Stew', 2: 'Cookies', 3: 'Nothing', 4: 'Ice Cream'}


## Making Transitional Matrix


In [ ]:
# fix matrix does not add up to one for some reason grrr
transition_matrix = np.array([[0.4, 0.0, 0.3, 0.0, 0.3], [0.0, 0.0, 0.1, 0.8, 0.1], [0.0, 0.0, 0.8, 0.1, 0.1], [0.2, 0.7, 0.05, 0.0, 0.05], [0.2, 0.0, 0.1, 0.7, 0.0]])
print(transition_matrix)

[[0.4  0.   0.3  0.   0.3 ]
 [0.   0.   0.1  0.8  0.1 ]
 [0.   0.   0.8  0.1  0.1 ]
 [0.2  0.7  0.05 0.   0.05]
 [0.2  0.   0.1  0.7  0.  ]]


##Random Walk on Markov Chain

In [ ]:
n = 15
start_state = 0
print(state[start_state], "--->", end=" ")
previous_state = start_state

# figure out how to normalize(?)
while n - 1:
  current_state = np.random.choice([0, 1, 2, 3, 4], p=transition_matrix[previous_state])
  print(state[current_state], "--->", end=" ")
  previous_state = current_state
  n -= 1

print("Stop")


Pizza ---> Pizza ---> Cookies ---> Cookies ---> Cookies ---> Cookies ---> Cookies ---> Cookies ---> Cookies ---> Nothing ---> Stew ---> Nothing ---> Stew ---> Nothing ---> Stew ---> Stop


## Following Core Algorithms Tutorial

### Creating the Distributions

In [ ]:
tfd = tfp.distributions

initial_distribution = tfd.Categorical(probs=[0.0, 0.0, 0.0, 1.0, 0.0])
transition_distribution = tfd.Categorical(probs=[[0.3, 0.0, 0.35, 0.0, 0.35], [0.0, 0.0, 0.1, 0.8, 0.1], [0.0, 0.0, 0.8, 0.1, 0.1], [0.2, 0.7, 0.05, 0.0, 0.05], [0.2, 0.0, 0.1, 0.7, 0.0]])
observation_distribution = tfd.Normal(loc=[30., 100., 10., 0., 15], scale=[0., 0., 0., 0., 0.])

###Creating the Model

In [ ]:
model = tfd.HiddenMarkovModel(
    initial_distribution = initial_distribution,
    transition_distribution = transition_distribution,
    observation_distribution = observation_distribution,
    num_steps = 100
)

###Running the Model

In [ ]:
mean = model.mean()

with tf.compat.v1.Session() as sess:
  print(mean.numpy())

[ 0.       77.25001   6.125    50.365005 15.91     37.024403 21.017763
 30.876324 23.471283 28.059067 24.633934 26.767431 25.182924 26.174826
 25.44171  25.902784 25.563541 25.777838 25.620836 25.720425 25.64776
 25.694033 25.660398 25.681896 25.666325 25.67631  25.669104 25.673738
 25.6704   25.672556 25.671007 25.672009 25.671295 25.67176  25.671429
 25.671644 25.67149  25.671593 25.671522 25.671566 25.671534 25.671555
 25.671541 25.67155  25.671543 25.67155  25.671547 25.67155  25.671547
 25.67155  25.671547 25.67155  25.671547 25.67155  25.671547 25.67155
 25.671547 25.67155  25.671547 25.67155  25.671547 25.67155  25.671547
 25.67155  25.671547 25.67155  25.671547 25.67155  25.671547 25.67155
 25.671547 25.67155  25.671547 25.67155  25.671547 25.67155  25.671547
 25.67155  25.671547 25.67155  25.671547 25.67155  25.671547 25.67155
 25.671547 25.67155  25.671547 25.67155  25.671547 25.67155  25.671547
 25.67155  25.671547 25.67155  25.671547 25.67155  25.671547 25.67155
 25.671547 

##Monte Carlo

In [ ]:
# selects a random choice among all the states in the transition matrix
# based on the probability of each choice
# runs n times
# the higher the value of n, the more accurate the result
# Uses the "law of averages" which states that an event will occur over periods of time
# at a frequency similar to its probability
# all the randomness 'averages out'
steps = 10**6 #1 million/1,000,000
start_state = 0
current_state = start_state
pi = np.array([0, 0, 0, 0, 0])
pi[start_state] = 1

i = 0

# fix error
while i < steps:
  current_state = np.random.choice([0, 1, 2, 3, 4], p=transition_matrix[current_state])
  pi[current_state] += 1
  i += 1

print("π = ", pi/steps)

## understand monte carlo approach
# make comments

π =  [0.115116 0.171852 0.36765  0.244762 0.100621]


## Repeated Matrix Multiplication

In [ ]:
steps = 10**3
matrix = transition_matrix
i = 0
while i < steps:
  matrix = np.matmul(matrix, transition_matrix)
  i += 1

print("Transition Matrix^n = \n", matrix, "\n")
print("π = ", matrix[0])

# make comments for this as well

Transition Matrix^n = 
 [[0.11495422 0.17090539 0.36927772 0.24415056 0.10071211]
 [0.11495422 0.17090539 0.36927772 0.24415056 0.10071211]
 [0.11495422 0.17090539 0.36927772 0.24415056 0.10071211]
 [0.11495422 0.17090539 0.36927772 0.24415056 0.10071211]
 [0.11495422 0.17090539 0.36927772 0.24415056 0.10071211]] 

π =  [0.11495422 0.17090539 0.36927772 0.24415056 0.10071211]


##Finding Left Eigen Vectors

In [ ]:
import scipy.linalg

print(transition_matrix)

values, left = scipy.linalg.eig(transition_matrix, right=False, left=True)

print("Left Eigen Vector values = \n", left, "\n")
print("Eigen Values = \n", values)

# fix data normalization??

# normalized dataVVV
pi = left[:,0]
pi_normalized = [x/np.sum(pi) for x in pi]
print(pi_normalized)

[[0.4  0.   0.3  0.   0.3 ]
 [0.   0.   0.1  0.8  0.1 ]
 [0.   0.   0.8  0.1  0.1 ]
 [0.2  0.7  0.05 0.   0.05]
 [0.2  0.   0.1  0.7  0.  ]]
Left Eigen Vector values = 
 [[ 0.14616112 -0.28769345  0.230582   -0.71094378 -0.25335955]
 [ 0.69821762 -0.64749369  0.34281217  0.26522947 -0.29618703]
 [-0.04579537  0.07425605  0.74071915  0.56365603  0.8764702 ]
 [-0.69098003  0.18361614  0.48973167  0.16413116 -0.2784458 ]
 [-0.10760334  0.67731495  0.20201431 -0.28207288 -0.04847782]] 

Eigen Values = 
 [-0.69274393+0.j -0.19850587+0.j  1.        +0.j  0.43317892+0.j
  0.65807088+0.j]
[175533638610046.1, 838531358595145.5, -54998403513217.086, -829839302602577.5, -129227291089396.06]


## Probability of a Chain Occuring

In [ ]:
def find_probability(sequence, transition_matrix, pi):
  start_state = sequence[0]
  probability = pi[start_state]
  previous_state, current_state = start_state, start_state
  for i in range(1, len(sequence)):
    curr_state = sequence[i]
    probability *= transition_matrix[previous_state][current_state]
    previous_state = current_state
  return probability

print("Finding probability of [1, 2, 3, 0]: ", find_probability([1, 2, 3, 0], transition_matrix, pi_normalized))


Finding probability of [1, 2, 3, 0]:  0.0
